In [1]:
import os
os.environ["GDAl_DATA"] = "venv\Lib\site-packages\osgeo\data"
os.environ["PROJ_LIB"] = "venv\Lib\site-packages\osgeo\data\proj"

In [2]:
import rasterio as rio
from rasterio.plot import show
from osgeo import gdalconst
from osgeo import gdal, osr
from osgeo_utils import gdal_calc
import matplotlib.pyplot as plt
import json
import numpy as np
from process_engine.oat import OAT
import pandas as pd

#### **Preparation**


In [4]:
# Constansts

# Image_to_attribute_Mapping
crit_dict = {
    "POPN": "population",
    "MARK": "market",
    "HLTH": "health",
    "ELEV": "elev",
    "SLOP": "slope",
    "LAND": "land_use",
    "RIVR": "river",
    "FLHZ": "flood_hazard",
    "ROADS": "road",
}

In [ ]:
# Organise Image Files

data_folder = os.path.join(os.getcwd(), "raw_data", "normalised_raster_v2")
imgs = [os.path.join(data_folder, x)
        for x in os.listdir(data_folder) if x.endswith(".tif")]

target_img_dict = {}
for x, y in crit_dict.items():
    for path in imgs:
        if (path.find(y) == -1):
            continue

        if (x in target_img_dict):
            if (path.endswith('v2.tif')):
                target_img_dict[y] = path
        else:
            target_img_dict[y] = path

In [6]:
# Organise Weights
weights = {}
with open('ahp_report.json', 'r') as ahp_rep:
    data = json.load(ahp_rep)
    for x, y in crit_dict.items():
        for z in data['target_weights']:
            if (z.startswith(y)):
                weights[y] = data['target_weights'][z]

#### **Adjustment Matrices**


In [7]:
adjust_model = OAT(weights)
adjust_matrx = adjust_model.solve(100, -100, 25)

[-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]


In [ ]:
# Load and Clean img matrices
# def loadImgMatrix(url, band=1):
#     output = {}
#     with gdal.Open(url) as ds:
#         array = ds.GetRasterBand(band).ReadAsArray()
#         output['array'] = array
#         output['bands'] = band
#         output['proj'] = ds.GetProjection()
#         output['transform'] = ds.GetGeoTransform()
#         output['ysize'] = array.shape[0]
#         output['xsize'] = array.shape[1]
#         output

#     return output


# def cleanImgArray(input):
#     array = input['array']
#     nan_clean = np.where((array < 0), 0, array)
#     input['array'] = nan_clean
#     return input


cleaned_imgs = {}

# for idx, url in target_img_dict.items():
#     _loaded = loadImgMatrix(url)
#     _cleaned = cleanImgArray(_loaded)
#     cleaned_imgs[idx] = _cleaned


for x, y in crit_dict.items():
    for path in imgs:
        if (path.find(y) == -1):
            continue

        cleaned_imgs[y] = path

cleaned_imgs_arrs = {x: y['array'] for x, y in cleaned_imgs.items()}

In [27]:
gdal.UseExceptions()


def clipRaster(raster_out_path: str,  raster_src_path: str, **kwargs):
    gdal.Warp(destNameOrDestDS=raster_out_path, srcDSOrSrcDSTab=raster_src_path,
              options=gdal.WarpOptions(**kwargs))

    if (os.path.exists(raster_out_path)):
        return raster_out_path
    else:
        return None

In [35]:
clipped_output_path = os.path.join(data_folder, "clipped")
clipped_paths = {}
if (not os.path.exists(clipped_output_path)):
    os.mkdir(clipped_output_path)

for idx, url in target_img_dict.items():
    raster_out_path = os.path.join(clipped_output_path, f"{idx}_norm_clip.tif")
    with gdal.Open(url) as ds:
        info = gdal.Info(ds)
        result = clipRaster(raster_out_path,
                            ds,
                            format="Gtiff",
                            dstSRS="EPSG:20936",
                            outputBounds="633665.0182967459550127 8153833.6840349500998855 725742.7175940889865160 8259549.5028867702931166",
                            cutlineDSName=os.path.join(
                                os.getcwd(), "raw_data", "vector_database.gpkg"),
                            cutlineLayer="chikwawa_bnd\'",
                            cropToCutline=True,
                            dstNodata=np.nan
                            )
        if (type(result) == str):
            clipped_paths[idx] = result

In [36]:
clipped_paths

{'population': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\population_norm_clip.tif',
 'market': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\market_norm_clip.tif',
 'health': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\health_norm_clip.tif',
 'elev': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\elev_norm_clip.tif',
 'slope': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\slope_norm_clip.tif',
 'land_use': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\land_use_norm_clip.tif',
 'river': 'c:\\Users\\User\\Documents\\Code\\Python\\research_unza\\raw_data\\normalised_raster_v2\\clipped\\river_norm_clip.tif',
 'flood_hazard': 'c:\\Users\\User\\Documents\\Code\\Python\\resea

In [18]:
# loop adjust to imgs arrys
# criterion indeces
crit_idx = {x: y for x, y in enumerate(adjust_model.ids)}

CRITERION_INDEX = 0
MAIN_CRITERION = crit_idx[CRITERION_INDEX]

target_adjust_mtrx = adjust_matrx[CRITERION_INDEX]

In [22]:
crit_idx

{0: 'population',
 1: 'market',
 2: 'health',
 3: 'elev',
 4: 'slope',
 5: 'land_use',
 6: 'river',
 7: 'flood_hazard',
 8: 'road'}

In [ ]:
output = {}

for y, row in enumerate([-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]):
    _y_out = {}
    for x, col in enumerate(target_adjust_mtrx[y]):
        _y_out[crit_idx[x]] = cleaned_imgs_arrs[crit_idx[x]] * col

    output[row] = _y_out

In [37]:
output_folder_path = os.path.join(os.getcwd(), 'output')

if (not os.path.exists(output_folder_path)):
    os.mkdir(output_folder_path)

for y, row in enumerate([-0.75, -0.5, -0.25, 0.0, 0.25, 0.5, 0.75]):

    calc = ""
    args = {}
    for x, col in enumerate(target_adjust_mtrx[y]):
        if (x == 0):
            calc += f"({crit_idx[x]}*{col})"
        else:
            calc += f"+({crit_idx[x]}*{col})"

        args[crit_idx[x]] = clipped_paths[crit_idx[x]]

    gdal_calc.Calc(calc, os.path.join(
        output_folder_path, f"{MAIN_CRITERION}_{row}.tif"), NoDataValue=0, **args)

Exception: Error! Dimensions of file c:\Users\User\Documents\Code\Python\research_unza\raw_data\normalised_raster_v2\clipped\market_norm_clip.tif (3069, 3523) are different from other files (3040, 3491).  Cannot proceed